In [1]:
#coding:utf-8
import networkx as nx
from networkx.algorithms import bipartite
import pandas as pd
import numpy as np
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import os
import cn2an
import chinese2digits as c2d
import operator
import pickle
import plotly.express as px
import plotly
import plotly.graph_objects as go
import plotly as py
import plotly.offline as offline
offline.init_notebook_mode(connected=True)
from urllib.request import urlopen
import json
import requests
from plotly.subplots import make_subplots
from collections import Counter

In [2]:
from ipynb.fs.full.case_to_graph_only_method import convert_csv_graph, provinces_map
from ipynb.fs.full.common_build_csv_pd import csv_graph, plaintiffs, defendants
all_cases = pd.read_csv('/Users/starice/OwnFiles/cityu/RA/case_study/data/total_extracted_result/all_cases.csv', encoding="utf-8")
print(all_cases['case_id'].drop_duplicates())
mapbox_access_token = "pk.eyJ1Ijoic3RhcmljZSIsImEiOiJjazN6Y2s5dTUxY2R6M2xxcHllbXk4YWFzIn0.lActFqLzqRWGn7dqr4BShw"
px.set_mapbox_access_token(mapbox_access_token)

0        57ab9058c2265c28a560195d
2        57baba28c2265c5f452d2cef
4        57abb74ac2265c258984430a
5        57ad1fe7c2265c04d1192a41
7        57baba99c2265c5f452d3090
                   ...           
63374    5ef091cbeb4b7e104f6a9bc8
63375    5ee72ae048198c22518af1b3
63379    5eed769ac5fb8255c9bfb343
63380    5eed661bc5fb8255c9bfa0d3
63381    5eed7836c5fb8255c9bfb4f1
Name: case_id, Length: 39121, dtype: object


In [3]:
base_url = "/Users/starice/Desktop/total_extracted_result/"
pre_dir = ['type1', 'type2', 'type3', 'type4']
dir_name = ['2014', '2015', '2016', '2017', '2018', '2019', '2020']
dir_sname = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
# csv_graph, _new_csvpd = convert_csv_graph(pre_dir[:], dir_name[:], dir_sname[:])
csv_graph = nx.read_gpickle("/Users/starice/Desktop/csv_graph.gpickle")

<h2>看一下律师的整体分布</h2>

In [18]:
temp_cases = all_cases.dropna(subset=['lawyer'])
lawyer_cases = temp_cases.groupby('year')['lawyer'].nunique().reset_index().rename(columns={"lawyer":"lawyer_count"}).merge(all_cases.groupby('year')['case_id'].nunique().reset_index().rename(columns={"case_id":"case_count"}))
lawyer_cases['lawyer_proportion'] = lawyer_cases['lawyer_count'] / lawyer_cases['case_count']
lawyer_cases

,year,lawyer_count,case_count,lawyer_proportion
0,2014,242,729,0.331962
1,2015,838,2821,0.297058
2,2016,1402,5607,0.250045
3,2017,2441,13892,0.175713
4,2018,1655,8245,0.200728
5,2019,1467,7547,0.194382
6,2020,127,280,0.453571


In [27]:
fig = px.bar(
    lawyer_cases, 
    x = 'year', 
    y = ['lawyer_count', 'case_count']
)
fig.add_trace(
    go.Scatter(
        x = lawyer_cases['year'], 
        y = lawyer_cases['lawyer_proportion'], 
        mode = "lines", 
        name = "lawyer_proportion", 
        showlegend=True,
    )
)
fig.update_layout(
    barmode = "group",
    title = "Temporal Distribution of Lawyer Amount"
    
)
fig.update_yaxes(type="log")
fig.write_html("/Users/starice/OwnFiles/cityu/RA/case_study/case_study_result/figures/temp_dist_lawyer.html")
fig.show()

In [4]:
#获取所有一审案件
first_cases = all_cases[all_cases['procedure']=="一审"]
print("一审案件数量： ", len(first_cases['case_id'].drop_duplicates()))

# 获取所有案件原告的节点度并排序
degree_1stplaintiffs = first_cases.groupby("plaintiff")['case_id'].unique().reset_index()
degree_1stplaintiffs['case_count'] = degree_1stplaintiffs['case_id'].apply(lambda r: len(r))
degree_1stplaintiffs.sort_values(by="case_count", inplace=True, ascending=False)
fps_200 = degree_1stplaintiffs[:200]

一审案件数量：  32322


In [8]:
new_selected_1stcases = first_cases[first_cases['plaintiff'].isin(fps_200['plaintiff'])]
new_selected_1stcases = new_selected_1stcases.drop(new_selected_1stcases[new_selected_1stcases['plaintiff'].isin(['XX', 'xx', '**'])].index)
new_selected_1stcases = new_selected_1stcases.groupby(['plaintiff', 'lawyer', 'province', 'city'])['case_id'].nunique().reset_index()
new_selected_1stcases['relationship'] = new_selected_1stcases[['plaintiff', 'lawyer']].apply(tuple, axis=1)
new_selected_1stcases.rename(columns={"case_id": "case_count"}, inplace=True)
new_selected_1stcases.head()

,plaintiff,lawyer,province,city,case_count,relationship
0,丛李松,刘新武,黑龙江省,哈尔滨市,1,"(丛李松, 刘新武)"
1,丛李松,刘金祥,黑龙江省,哈尔滨市,4,"(丛李松, 刘金祥)"
2,丛李松,吴迪,黑龙江省,哈尔滨市,1,"(丛李松, 吴迪)"
3,丛李松,张雪,黑龙江省,哈尔滨市,2,"(丛李松, 张雪)"
4,丛李松,徐秋敏,黑龙江省,哈尔滨市,7,"(丛李松, 徐秋敏)"


In [9]:
pl_graph = nx.Graph()
pl_graph.add_nodes_from(list(new_selected_1stcases['plaintiff'].drop_duplicates()), bipartite=0)
pl_graph.add_nodes_from(list(new_selected_1stcases['lawyer'].dropna()), bipartite=1)
relationship = list(new_selected_1stcases['relationship'])
case_counts = list(new_selected_1stcases['case_count'])
cities = list(new_selected_1stcases['city'])
provinces = list(new_selected_1stcases['province'])
pl_graph.add_edges_from([(relationship[i][0], relationship[i][1], \
                                 {'province': provinces[i], 'city': cities[i]})
                                for i in range(len(relationship))])
for i in range(len(relationship)):
    pl_graph[relationship[i][0]][relationship[i][1]]['weight'] = case_counts[i]

In [10]:
# save graph to text file
nx.write_edgelist(pl_graph, "/Users/starice/Desktop/plaintiff_lawyer.txt", delimiter=' ', data=['weight'])

In [11]:
# save graph to pickle file
nx.write_gpickle(pl_graph, "/Users/starice/Desktop/plaintiff_lawyer.gpickle")

In [12]:
graph = nx.read_gpickle("/Users/starice/Desktop/plaintiff_lawyer.gpickle")
graph['丛李松']

AtlasView({'刘新武': {'province': '黑龙江省', 'city': '哈尔滨市', 'weight': 1}, '刘金祥': {'province': '黑龙江省', 'city': '哈尔滨市', 'weight': 4}, '吴迪': {'province': '黑龙江省', 'city': '哈尔滨市', 'weight': 1}, '张雪': {'province': '黑龙江省', 'city': '哈尔滨市', 'weight': 2}, '徐秋敏': {'province': '黑龙江省', 'city': '哈尔滨市', 'weight': 7}, '李新宏': {'province': '黑龙江省', 'city': '哈尔滨市', 'weight': 1}, '李艳丽': {'province': '黑龙江省', 'city': '哈尔滨市', 'weight': 3}, '梁伟': {'province': '黑龙江省', 'city': '哈尔滨市', 'weight': 1}, '董彬': {'province': '黑龙江省', 'city': '哈尔滨市', 'weight': 1}, '赵宏文': {'province': '黑龙江省', 'city': '哈尔滨市', 'weight': 2}, '赵艳凤': {'province': '黑龙江省', 'city': '哈尔滨市', 'weight': 1}, '金玉莹': {'province': '黑龙江省', 'city': '哈尔滨市', 'weight': 2}, '闫玉香': {'province': '黑龙江省', 'city': '哈尔滨市', 'weight': 1}, '高振娟': {'province': '黑龙江省', 'city': '哈尔滨市', 'weight': 1}, '高晓秋': {'province': '黑龙江省', 'city': '哈尔滨市', 'weight': 2}})

In [13]:
# 看一下原告和律师组合的赢率是否会增强，是否惩罚金偏高
all_cases['is_lawyer_exist'] = all_cases['lawyer'].apply(lambda x: True if x is not np.nan else False)
temp_first_cases = all_cases[all_cases['procedure']=="一审"]
lawyer_case_cc = temp_first_cases.groupby('is_lawyer_exist')['case_id'].nunique().reset_index()
lawyer_case_cc.rename(columns={"case_id": "case_count"}, inplace=True)
lawyer_case_sc = temp_first_cases[temp_first_cases['is_success']=="TRUE"].groupby('is_lawyer_exist')['case_id'].nunique().reset_index()
lawyer_case_sc.rename(columns={"case_id": "success_count"}, inplace=True)
lawyer_case_sc.fillna(0, inplace=True)
# lawyer_case_penalty = temp_first_cases.groupby('is_lawyer_exist')['penalty'].mean().reset_index()
lawyer_case = lawyer_case_cc.merge(lawyer_case_sc, how="left")\
# .merge(lawyer_case_penalty, how="left")
lawyer_case['success_rate'] = lawyer_case['success_count'] / lawyer_case['case_count']
lawyer_case.head()

,is_lawyer_exist,case_count,success_count,success_rate
0,False,20262,18006,0.888659
1,True,12183,9786,0.803250


In [14]:
fig = px.box(
    temp_first_cases[temp_first_cases['is_success']=="TRUE"].replace(0, pd.np.nan).dropna(subset=['penalty']).sort_values(by='year'), 
    x="is_lawyer_exist", 
    y="penalty")
fig.update_yaxes(type="log")
fig.show()
# 有律师的案件普遍惩罚金要高于没有律师的案件

<ipython-input-14-1e13aadacc42>:2: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead



In [15]:
#看一下标的和律师的联系
fig = px.box(
    temp_first_cases.replace(0, pd.np.nan).dropna(subset=['objectmoney']).sort_values(by='year'), 
    x="is_lawyer_exist", 
    y="objectmoney")
fig.update_yaxes(type="log")
fig.show()

<ipython-input-15-15193225bc19>:3: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead



In [16]:
# 是否有律师的案件标的越高？ 
# 看一下逻辑回归的结果
logistic_om_lw = temp_first_cases.replace(0, pd.np.nan).dropna(subset=['objectmoney'])[['case_id', 'objectmoney', 'is_lawyer_exist']]

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
X = logistic_om_lw[['objectmoney']].values
y = logistic_om_lw['is_lawyer_exist'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# clf = LogisticRegression(random_state=0).fit(X_train, y_train)
clf = LogisticRegressionCV(cv=5, random_state=0).fit(X_train, y_train)
# clf.score(X_test, y_test)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

<ipython-input-16-724d1d6a9461>:2: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning:

lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning:

lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternati

              precision    recall  f1-score   support

       False       0.59      0.94      0.73      3870
        True       0.75      0.20      0.32      3128

    accuracy                           0.61      6998
   macro avg       0.67      0.57      0.52      6998
weighted avg       0.66      0.61      0.55      6998



/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning:

lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning:

lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_log

In [17]:
# y：赢率，惩罚金（律师作用不大），法院愿不愿意让被告付惩罚金
# 原告律师的团体其罚金会不会高，标的大不大，赢率如何？针对的被告有没有什么特点？

In [18]:
# louvain bipartite graph community detection(unfinished!)
# 如何将聚类的结果显示在python上呢？
import leidenalg as la
import matplotlib.pyplot as plt
import igraph as ig

In [19]:
G = ig.Graph.from_networkx(graph)
G.is_bipartite()

False